In [ ]:
import batoid
import os
import yaml
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LSST_r_fn = os.path.join(batoid.datadir, "LSST", "LSST_r2.yaml")
config = yaml.load(open(LSST_r_fn))
fiducial_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
def perturb(M1M3_dx=0, M1M3_dy=0,
            M2_dx=0, M2_dy=0, M2_dz=0,
            cam_dx=0, cam_dy=0, cam_dz=0):
    telescope = fiducial_telescope
    if not (M1M3_dx == M1M3_dy == 0):
        telescope = telescope.withGloballyShiftedOptic('LSST.M1', batoid.Vec3(M1M3_dx, M1M3_dy, 0))
        telescope = telescope.withGloballyShiftedOptic('LSST.M3', batoid.Vec3(M1M3_dx, M1M3_dy, 0))        
    if not (M2_dx == M2_dy == 0):
        telescope = telescope.withGloballyShiftedOptic('LSST.M2', batoid.Vec3(M2_dx, M2_dy, 0))
    if not (cam_dx == cam_dy == cam_dz == 0):
        telescope = telescope.withGloballyShiftedOptic('LSST.LSSTCamera', batoid.Vec3(cam_dx, cam_dy, cam_dz))
    return telescope

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1100.0,step=10.0,value=620.0),
          theta_x=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=-1.40),
          theta_y=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=0.0),
          M1M3_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          M1M3_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dz=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=1, max=3, step=0.1, value=1))
def spot(wavelen, theta_x, theta_y, M1M3_dx, M1M3_dy, M2_dx, M2_dy, M2_dz, cam_dx, cam_dy, cam_dz, logscale):
    """Display a spot diagram for LSST.

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param M1M3_dx  M1M3 x decenter in mm
    @param M1M3_dy  M1M3 y decenter in mm
    @param M2_dx    M2 x decenter in mm
    @param M2_dy    M2 y decenter in mm
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic axes zoom level
    """
    telescope = perturb(M1M3_dx*1e-3, M1M3_dy*1e-3,
                        M2_dx*1e-3, M2_dy*1e-3, M2_dz*1e-3,
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    xcos = np.sin(theta_x*np.pi/180)
    ycos = np.sin(theta_y*np.pi/180)
    zcos = -np.sqrt(1.0 - xcos**2 - ycos**2)
    rays = batoid.circularGrid(18.0, 6.0, 0.5, xcos, ycos, zcos,
                               128, 512, wavelen*1e-9, batoid.Air())

    rs, _ = telescope.trace(rays)
    rs = batoid.trimVignetted(rs)
    spots = np.vstack([rs.x, rs.y]).T
    spots = np.array(spots)
    spots -= np.mean(spots, axis=0)
    spots *= 1e6 # meters -> microns
    plt.figure(figsize=(4.5,4))
    plt.scatter(spots[:,0], spots[:,1], s=1, alpha=0.5)
    plt.xlim(-10**logscale, 10**logscale)
    plt.ylim(-10**logscale, 10**logscale)
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}$".format(theta_x, theta_y))
    plt.xlabel("microns")
    plt.ylabel("microns")

In [ ]:
#  http://stackoverflow.com/a/18968498
def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trailing dimensions
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1100.0,step=10.0,value=620.0),
          theta_x=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=-1.4),
          theta_y=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=0.0),
          M1M3_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          M1M3_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dz=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          contrast=widgets.FloatSlider(min=-6, max=-4, step=0.1, value=-5.5))
def opd(wavelen, theta_x, theta_y, M1M3_dx, M1M3_dy, M2_dx, M2_dy, M2_dz, cam_dx, cam_dy, cam_dz, contrast):
    """Display optical path differences

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param M1M3_dx  M1M3 x decenter in mm
    @param M1M3_dy  M1M3 y decenter in mm
    @param M2_dx    M2 x decenter in mm
    @param M2_dy    M2 y decenter in mm
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param contrast Logarithmic colorbar contrast level
    """
    telescope = perturb(M1M3_dx*1e-3, M1M3_dy*1e-3, 
                        M2_dx*1e-3, M2_dy*1e-3, M2_dz*1e-3,
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    xcos = np.sin(theta_x*np.pi/180)
    ycos = np.sin(theta_y*np.pi/180)
    zcos = -np.sqrt(1.0 - xcos**2 - ycos**2)
    rays = batoid.circularGrid(18.0, 6.0, 0.5, xcos, ycos, zcos,
                               128, 512, wavelen*1e-9, batoid.Air())
    rs, _ = telescope.trace(rays)
    rs = batoid.trimVignetted(rs)

    theta_opd = np.vstack([rs.vx, rs.vy, rs.t0]).T    
    opd = theta_opd[:,2]
    opd[:] -= np.mean(opd)    
    x = theta_opd[:,0]
    y = theta_opd[:,1]
    p, n = planeFit(theta_opd[::10,:].T)
    const = np.dot(p, n)
    opd[:] -= (const-n[0]*x-n[1]*y)/n[2]
    plt.figure(figsize=(5.3,4))
    plt.scatter(x, y, c=opd, s=5, vmin=-10**contrast, vmax=10**contrast)
    plt.xlim(-0.5, 0.5)
    plt.ylim(-0.5, 0.5)
    plt.axhline(0.0, c='k')
    plt.axvline(0.0, c='k')
    plt.xlabel("vx")
    plt.ylabel("vy")
    plt.colorbar()